In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel('./QUTZS_final.xlsx')
# print(df.head())
# print(df.info())
# print(df.shape)


In [6]:
df.columns

Index(['Label', 'pkt arrival time_sv1', 'MACsrc_sv1', 'MACdst_sv1',
       'APPID_sv1', 'SVlength_sv1', 'noASDU_sv1', 'svID1_sv1', 'smpCnt1_sv1',
       'Data1_sv1',
       ...
       'datset_GOOSE3', 'goID_GOOSE3', 't_GOOSE3', 'stNum_GOOSE3',
       'sqNum_GOOSE3', 'simulation_GOOSE3', 'confRev_GOOSE3', 'ndsCom_GOOSE3',
       'num of data_GOOSE3', 'data_GOOSE3'],
      dtype='object', length=139)

In [2]:
pwd

'/home/csu/Liyi/git_space/QUT-ZSS-2023-SV/Datasets/Processed dataset (supervised ML)'

In [ ]:
import os
from pathlib import Path



root_dir = Path("./Benign-behaviour-only")

print("Searching in:", root_dir)

# 忽略大小写搜索 xlsx
xlsx_files = list(root_dir.rglob("*QUTZS_final.xlsx"))

print(f"Found {len(xlsx_files)} files.")

for xlsx_path in xlsx_files:
    try:
        print("\nProcessing:", xlsx_path)
        df = pd.read_excel(xlsx_path)
        final_df = generate_final_df(df)

        # 输出文件名，如 QUTZS_final_ics_logs.csv
        out_path = xlsx_path.with_name(xlsx_path.stem + "_ics_logs.csv")

        final_df.to_csv(out_path, index=False, encoding="utf-8")
        print("Saved:", out_path)

    except Exception as e:
        print(f"Failed {xlsx_path}: {e}")



### 加上历史数据

In [36]:
import pandas as pd
import re
import numpy as np
import json
from typing import List, Dict, Any
from pathlib import Path


def extract_groups(columns):
    groups = set()
    for col in columns:
        m = re.search(r"(sv\d+|SV\d+|goose\d+|GOOSE\d+)", col)
        if m:
            groups.add(m.group().lower())
    return list(groups)

def format_single_group(row: pd.Series, g: str, columns: List[str]) -> str:
    """
    格式化单个 GOOSE 或 SV 数据组，确保时间戳和关键字段被包含。
    """
    # --- 1. 查找时间戳字段 ---
    if "goose" in g.lower():
        timestamp_col = next((c for c in columns if c.lower().startswith('t_goose') and g in c.lower()), None)
    elif "sv" in g.lower():
        timestamp_col = next((c for c in columns if 'pkt arrival time' in c.lower() and g in c.lower()), None)
    else:
        timestamp_col = None
        
    macsrc_col    = next((c for c in columns if ("src" in c.lower()) and g in c.lower()), None)
    macdst_col    = next((c for c in columns if ("dst" in c.lower()) and g in c.lower()), None)
    appid_col     = next((c for c in columns if "appid" in c.lower() and g in c.lower()), None)
    stnum_col     = next((c for c in columns if "stnum" in c.lower() and g in c.lower()), None)
    sqnum_col     = next((c for c in columns if "sqnum" in c.lower() and g in c.lower()), None)


    # --- 2. 字段值提取 ---
    timestamp = row.get(timestamp_col, "N/A")
    macsrc    = row.get(macsrc_col, "N/A")
    appid     = row.get(appid_col, "N/A")
    stnum     = row.get(stnum_col, None)
    sqnum     = row.get(sqnum_col, None)
    
    
    msg_type = "SV" if "sv" in g.lower() else "GOOSE"
    
    base = f"{msg_type}_PKT (ts={timestamp}, src={macsrc}, APPID={appid}"

    # GOOSE 特有字段
    if msg_type == "GOOSE":
        if pd.notna(stnum):
            base += f", stNum={int(stnum)}"
        if pd.notna(sqnum):
            base += f", sqNum={int(sqnum)}"
    
    additional = []
    ignore_cols = {timestamp_col, macsrc_col, macdst_col, appid_col, stnum_col, sqnum_col}
    
    # --- 3. 提取额外数据字段 ---
    for col in columns:
        # 确保列名包含当前分组标记（例如 'sv1'），且不是基础忽略字段
        if g.lower() in col.lower() and col not in ignore_cols:
            val = row[col]
            if pd.notna(val):
                
                final_key_name = col.strip() 

                # 针对布尔/数值进行格式化 (保持不变)
                if isinstance(val, (bool, np.bool_)):
                    val_str = "True" if val else "False"
                elif isinstance(val, (float, np.floating)):
                    val_str = f"{val:.4f}"
                else:
                    val_str = str(val)
                    
                # 确保不包含时间戳或 MAC 等已在 base 中出现的字段。
                if not any(keyword in final_key_name.lower() for keyword in ["time", "ts", "src", "dst", "appid", "stnum", "sqnum", "pkt arrival"]):
                    additional.append(f"{final_key_name}={val_str}")

    if additional:
        base += ", " + ", ".join(additional)
    
    base += ")"
    return base

def row_to_single_log(row: pd.Series, columns: List[str], groups: List[str]) -> str:
    """
    生成一个时间步（一行数据）的所有事件序列，以分号分隔。
    移除顶层时间戳包装，时间戳已嵌入到每个 packet 中。
    """
    parts = []
    for g in groups:
        event = format_single_group(row, g, columns)
        parts.append(event)

    return "; ".join(parts)

def analyze_numeric_diffs(diff_list: List[float], is_time_or_counter: bool = False):
    """
    分析差分列表，返回语义标签。
    is_time_or_counter: 如果为 True，则应用针对时间/计数器的宽松规则。
    """
    semantics = []
    
    # 阈值逻辑 (保持不变)
    JUMP_THRESHOLD = 100 

    if not diff_list:
        return "steady"
    
    # 判断是否存在显著非零变化
    has_non_zero_diff = any(abs(d) > 1e-6 for d in diff_list)

    if not has_non_zero_diff:
        return "steady"

    # 1. 趋势判断 (宽松化：允许部分零值)
    is_mostly_increasing = all(d >= 0 for d in diff_list)
    is_mostly_decreasing = all(d <= 0 for d in diff_list)

    if is_mostly_increasing:
        semantics.append("increase")
    elif is_mostly_decreasing:
        semantics.append("decrease")
        
    if is_time_or_counter and is_mostly_increasing:
        # 针对时间/计数器，只要大部分时间在增加，就认为是正常自增
        return "incrementing" 
    
    # 2. 异常判断 (仅对非时间/计数器的物理量才判断大幅突变)
    # 如果是时间/计数器，跳过 JUMP_THRESHOLD 判断，除非阈值专门针对时间设计
    if not is_time_or_counter and any(abs(d) > JUMP_THRESHOLD for d in diff_list):
        semantics.append("abnormal_jump")
        
    # 3. 震荡判断 (符号翻转)
    if len(diff_list) > 1:
        # 忽略极小的浮点数变化
        sign_changes = sum(1 for i in range(1, len(diff_list)) 
                           if diff_list[i] * diff_list[i-1] < 0 and 
                           abs(diff_list[i]) > 1e-6 and abs(diff_list[i-1]) > 1e-6)
        
        if sign_changes > 0:
            semantics.append("fluctuation")

    semantics = list(set(semantics))
    if semantics:
        return semantics if len(semantics) > 1 else semantics[0]

    # 无法被定义为上述任何语义，则标记为复杂变化
    return "complex_change"

def generate_optimized(df, diff_window=3):
    labels = df["Label"] if "Label" in df.columns else [None]*len(df)
    df_no_label = df.drop(columns=["Label"], errors='ignore')
    columns = df_no_label.columns.tolist()
    numeric_cols = df_no_label.select_dtypes(include=[np.number]).columns
    TIME_COUNTER_COLS = [
        col for col in numeric_cols 
        if "time" in col.lower() or "ts" in col.lower() or "smpcnt" in col.lower() or "sqnum" in col.lower()
    ]
    groups = extract_groups(columns)

    df_diff = df_no_label[numeric_cols].diff().fillna(0)

    event_sequences = []
    diff_vectors = []

    # 转换为 dict 列表以便快速迭代
    records = df_no_label.to_dict('records')
    diff_records = df_diff.to_dict('records')
    
    # 用于存储最近 N 个 diff 值的 rolling buffer
    # 结构: {col_name: [diff_t-2, diff_t-1, diff_t]}
    rolling_diffs = {col: [] for col in columns}

    print(f"Processing {len(records)} rows...")

    for i, row in enumerate(records):
        # A. 生成文本 Log
        seq = row_to_single_log(row, columns, groups)
        event_sequences.append(seq)

        # B. 生成 Diff Vector
        row_diff_vec = {}
        current_diff_row = diff_records[i] if i < len(diff_records) else {}

        for col, val in row.items():
            # 跳过非关键列（如单纯的时间戳列，视情况而定）
            
            # Case 1: Boolean / String (状态量)
            # 使用原始值比较 (Current vs Previous)
            prev_val = records[i-1][col] if i > 0 else val
            
            if isinstance(val, (bool, str)) or (isinstance(val, (int, float)) and col not in numeric_cols):
                if val == prev_val:
                    sem = "state_hold"
                else:
                    if isinstance(val, bool) and isinstance(prev_val, bool):
                        sem = "trip_activation" if (not prev_val and val) else "state_drop"
                    else:
                        sem = "state_change"
                
                # 对于状态量，只记录语义
                if sem != "state_hold": 
                     row_diff_vec[col] = sem

            # Case 2: Numeric (模拟量/时间/计数器)
            elif col in numeric_cols:
                d_val = current_diff_row.get(col, 0.0)
                rolling_diffs[col].append(d_val)
                if len(rolling_diffs[col]) > diff_window:
                    rolling_diffs[col].pop(0)
                
                current_window_diffs = rolling_diffs[col]
                
                is_tc = col in TIME_COUNTER_COLS 
                sem = analyze_numeric_diffs(current_window_diffs, is_time_or_counter=is_tc)
                
                # 如果是时间/计数器，并且是 incrementing，只保留 sem
                if is_tc and sem == "incrementing":
                    row_diff_vec[col] = "incrementing" # 进一步压缩
                else:
                    # 对于物理量或非正常的时间/计数器变化，保留 delta 列表
                    formatted_diffs = [round(x, 3) for x in current_window_diffs]
                    row_diff_vec[col] = {
                        "delta": formatted_diffs, 
                        "sem": sem 
                    }
            
        diff_vectors.append(row_diff_vec)

    return pd.DataFrame({
        "Label": labels,
        "Event_sequence": event_sequences,
        "Diff_vector": diff_vectors
    })

def _json_safe(obj):
    if isinstance(obj, (np.integer, int)):
        return int(obj)
    if isinstance(obj, (np.floating, float)):
        return float(obj)
    if isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    return str(obj)

def main(input_path: str):
    try:
        if input_path.endswith('.csv'):
            df = pd.read_csv(input_path)
        else:
            df = pd.read_excel(input_path)
        print(f"Loaded data: {len(df)} rows.")
    except Exception as e:
        print(f"Error: {e}")
        return

    out_df = generate_optimized(df, diff_window=3)

    # 序列化
    p = Path(input_path)
    parent_name = p.parent.name or p.stem
    out_csv = f"{parent_name}.csv"
    print(f"Output file will be: {out_csv}")
    out_df["Diff_vector"] = out_df["Diff_vector"].apply(
        lambda d: json.dumps(d, ensure_ascii=False, default=_json_safe)
    )
    
    out_df.to_csv(out_csv, index=False)
    print(f"Success! Wrote to {out_csv}")
    
import os
from pathlib import Path
if __name__ == "__main__":   
    #main('./974/974d/QUTZS_final.xlsx')




    root_dir = Path("./Benign-behaviour-only")

    print("Searching in:", root_dir)

    # 忽略大小写搜索 xlsx
    xlsx_files = list(root_dir.rglob("*QUTZS_final.xlsx"))

    print(f"Found {len(xlsx_files)} files.")

    for xlsx_path in xlsx_files:
        print("\nProcessing:", xlsx_path)
        #df = pd.read_excel(xlsx_path)
        main(xlsx_path)


        

Searching in: Benign-behaviour-only
Found 11 files.

Processing: Benign-behaviour-only/105/QUTZS_final.xlsx
Error: 'PosixPath' object has no attribute 'endswith'

Processing: Benign-behaviour-only/108/QUTZS_final.xlsx
Error: 'PosixPath' object has no attribute 'endswith'

Processing: Benign-behaviour-only/104/QUTZS_final.xlsx
Error: 'PosixPath' object has no attribute 'endswith'

Processing: Benign-behaviour-only/0/QUTZS_final.xlsx
Error: 'PosixPath' object has no attribute 'endswith'

Processing: Benign-behaviour-only/109/QUTZS_final.xlsx
Error: 'PosixPath' object has no attribute 'endswith'

Processing: Benign-behaviour-only/102/QUTZS_final.xlsx
Error: 'PosixPath' object has no attribute 'endswith'

Processing: Benign-behaviour-only/107/QUTZS_final.xlsx
Error: 'PosixPath' object has no attribute 'endswith'

Processing: Benign-behaviour-only/101/QUTZS_final.xlsx
Error: 'PosixPath' object has no attribute 'endswith'

Processing: Benign-behaviour-only/103/QUTZS_final.xlsx
Error: 'PosixP